In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1772969,2021-10-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1772970,2021-10-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1772971,2021-10-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1772972,2021-10-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
46167,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46169,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46171,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46173,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46175,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1079520,2021-10-22,Arkansas,Arkansas,5001,3467,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1079522,2021-10-23,Arkansas,Arkansas,5001,3469,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1079524,2021-10-24,Arkansas,Arkansas,5001,3470,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1079526,2021-10-25,Arkansas,Arkansas,5001,3473,59.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1772969,2021-10-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1772970,2021-10-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1772971,2021-10-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1772972,2021-10-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-10-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
644,2021-10-26,Snohomish,Washington,64803,749.00,53061,WA,County,822083,7882.78,91.11
1286,2021-10-26,Cook,Illinois,638883,11804.00,17031,IL,County,5150233,12404.93,229.19
1927,2021-10-26,Orange,California,325252,5584.00,6059,CA,County,3175692,10241.93,175.84
2567,2021-10-26,Maricopa,Arizona,729597,12092.00,4013,AZ,County,4485414,16265.99,269.58
3207,2021-10-26,Los Angeles,California,1487318,26579.00,6037,CA,County,10039107,14815.24,264.75
...,...,...,...,...,...,...,...,...,...,...,...
1771581,2021-10-26,Wheeler,Oregon,109,1.00,41069,OR,County,1332,8183.18,75.08
1771960,2021-10-26,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00
1772308,2021-10-26,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10
1772652,2021-10-26,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
644,2021-10-26,Snohomish,Washington,64803,749.00,53061,WA,County,822083,7882.78,91.11,91.11,7882.78
1286,2021-10-26,Cook,Illinois,638883,11804.00,17031,IL,County,5150233,12404.93,229.19,229.19,12404.93
1927,2021-10-26,Orange,California,325252,5584.00,6059,CA,County,3175692,10241.93,175.84,175.84,10241.93
2567,2021-10-26,Maricopa,Arizona,729597,12092.00,4013,AZ,County,4485414,16265.99,269.58,269.58,16265.99
3207,2021-10-26,Los Angeles,California,1487318,26579.00,6037,CA,County,10039107,14815.24,264.75,264.75,14815.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771581,2021-10-26,Wheeler,Oregon,109,1.00,41069,OR,County,1332,8183.18,75.08,75.08,8183.18
1771960,2021-10-26,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00,0.00,5882.35
1772308,2021-10-26,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10,229.10,6643.76
1772652,2021-10-26,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1129147,2021-10-26,Hale,Alabama,3119,88.00,1065,AL,County,14651,21288.65,600.64,600.64,21288.65,1
1229867,2021-10-26,Clarke,Alabama,4772,84.00,1025,AL,County,23622,20201.51,355.60,355.60,20201.51,2
318860,2021-10-26,Calhoun,Alabama,22255,490.00,1015,AL,County,113605,19589.81,431.32,431.32,19589.81,3
781574,2021-10-26,Etowah,Alabama,19844,500.00,1055,AL,County,102268,19403.92,488.91,488.91,19403.92,4
952836,2021-10-26,Winston,Alabama,4538,103.00,1133,AL,County,23629,19205.21,435.91,435.91,19205.21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683672,2021-10-26,Weston,Wyoming,1047,9.00,56045,WY,County,6927,15114.77,129.93,129.93,15114.77,19
949941,2021-10-26,Johnson,Wyoming,1180,16.00,56019,WY,County,8445,13972.76,189.46,189.46,13972.76,20
1077797,2021-10-26,Sublette,Wyoming,1350,16.00,56035,WY,County,9831,13732.07,162.75,162.75,13732.07,21
1341604,2021-10-26,Lincoln,Wyoming,2720,20.00,56023,WY,County,19830,13716.59,100.86,100.86,13716.59,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
882081,2021-10-26,Lowndes,Alabama,1757,64.00,1085,AL,County,9726,18064.98,658.03,658.03,18064.98,18,1
1129147,2021-10-26,Hale,Alabama,3119,88.00,1065,AL,County,14651,21288.65,600.64,600.64,21288.65,1,2
373541,2021-10-26,Walker,Alabama,11255,370.00,1127,AL,County,63521,17718.55,582.48,582.48,17718.55,23,3
880921,2021-10-26,Crenshaw,Alabama,2454,75.00,1041,AL,County,13772,17818.76,544.58,544.58,17818.76,22,4
780993,2021-10-26,Dallas,Alabama,5013,202.00,1047,AL,County,37196,13477.26,543.07,543.07,13477.26,61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683672,2021-10-26,Weston,Wyoming,1047,9.00,56045,WY,County,6927,15114.77,129.93,129.93,15114.77,19,19
1307856,2021-10-26,Uinta,Wyoming,3805,26.00,56041,WY,County,20226,18812.42,128.55,128.55,18812.42,6,20
1341604,2021-10-26,Lincoln,Wyoming,2720,20.00,56023,WY,County,19830,13716.59,100.86,100.86,13716.59,22,21
878601,2021-10-26,Albany,Wyoming,6265,32.00,56001,WY,County,38880,16113.68,82.30,82.30,16113.68,16,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,91.11,7882.78,27,29
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,91.11,7882.78,27,29
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,91.11,7882.78,27,29
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,91.11,7882.78,27,29
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,91.11,7882.78,27,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768860,2021-10-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1768861,2021-10-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1768862,2021-10-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1768863,2021-10-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1124463,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21288.65,2,1,1.00,nan
1124464,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21288.65,2,1,0.00,nan
1124465,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21288.65,2,1,0.00,nan
1124466,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21288.65,2,1,0.00,nan
1124467,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,600.64,21288.65,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364328,2021-10-22,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11603.38,8,23,2.00,3.43
1364329,2021-10-23,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11603.38,8,23,0.00,3.43
1364330,2021-10-24,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11603.38,8,23,0.00,3.43
1364331,2021-10-25,Crook,Wyoming,874,18.00,56011,WY,County,7584,11524.26,237.34,237.34,11603.38,8,23,10.00,3.86


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
878549,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18064.98,1,18,1.00,4.64,0.00,nan
878550,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18064.98,1,18,0.00,3.50,0.00,nan
878551,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18064.98,1,18,0.00,2.93,0.00,nan
878552,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18064.98,1,18,0.00,2.86,0.00,nan
878553,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18064.98,1,18,0.00,2.64,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369404,2021-10-22,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,22008.18,23,2,9.00,9.64,0.00,0.07
369405,2021-10-23,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,22008.18,23,2,0.00,9.64,0.00,0.07
369406,2021-10-24,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,22008.18,23,2,0.00,9.64,0.00,0.07
369407,2021-10-25,Teton,Wyoming,5152,14.00,56039,WY,County,23464,21957.04,59.67,59.67,22008.18,23,2,33.00,10.86,0.00,0.07


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-10-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
446297,2021-10-26,Imperial,California,36262,765.00,6025,CA,County,181215,20010.48,422.15,422.15,20010.48,1,3,191.00,58.50,0.00,0.29
3207,2021-10-26,Los Angeles,California,1487318,26579.00,6037,CA,County,10039107,14815.24,264.75,264.75,14815.24,2,13,845.00,1013.93,27.00,16.64
221521,2021-10-26,San Bernardino,California,360071,5754.00,6071,CA,County,2180085,16516.37,263.93,263.93,16516.37,3,5,1394.00,328.50,34.00,14.07
98717,2021-10-26,Stanislaus,California,86874,1341.00,6099,CA,County,550660,15776.34,243.53,243.53,15776.34,4,7,136.00,164.86,3.00,2.79
78455,2021-10-26,San Joaquin,California,103316,1745.00,6077,CA,County,762148,13555.90,228.96,228.96,13555.90,5,18,540.00,138.57,0.00,4.14
783850,2021-10-26,Inyo,California,2217,41.00,6027,CA,County,18039,12290.04,227.29,227.29,12290.04,6,23,75.00,23.29,0.00,0.14
951620,2021-10-26,Kings,California,33325,335.00,6031,CA,County,152940,21789.59,219.04,219.04,21789.59,7,2,387.00,89.43,2.00,1.86
1078304,2021-10-26,Tuolumne,California,7178,118.00,6109,CA,County,54478,13175.96,216.60,216.60,13175.96,8,22,27.00,25.36,1.00,0.86
709953,2021-10-26,Merced,California,43167,588.00,6047,CA,County,277680,15545.59,211.75,211.75,15545.59,9,8,38.00,93.71,0.00,1.21
122514,2021-10-26,Tulare,California,81147,986.00,6107,CA,County,466195,17406.24,211.50,211.50,17406.24,10,4,772.00,208.79,8.00,3.14


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1671766,2021-10-26,Lassen,California,7543,50.00,6035,CA,County,30573,24672.10,163.54,163.54,24672.10,20,1,-44.00,16.07,1.00,0.50
951620,2021-10-26,Kings,California,33325,335.00,6031,CA,County,152940,21789.59,219.04,219.04,21789.59,7,2,387.00,89.43,2.00,1.86
446297,2021-10-26,Imperial,California,36262,765.00,6025,CA,County,181215,20010.48,422.15,422.15,20010.48,1,3,191.00,58.50,0.00,0.29
122514,2021-10-26,Tulare,California,81147,986.00,6107,CA,County,466195,17406.24,211.50,211.50,17406.24,10,4,772.00,208.79,8.00,3.14
221521,2021-10-26,San Bernardino,California,360071,5754.00,6071,CA,County,2180085,16516.37,263.93,263.93,16516.37,3,5,1394.00,328.50,34.00,14.07
278793,2021-10-26,Kern,California,147784,1652.00,6029,CA,County,900202,16416.76,183.51,183.51,16416.76,14,6,490.00,346.57,0.00,4.00
98717,2021-10-26,Stanislaus,California,86874,1341.00,6099,CA,County,550660,15776.34,243.53,243.53,15776.34,4,7,136.00,164.86,3.00,2.79
709953,2021-10-26,Merced,California,43167,588.00,6047,CA,County,277680,15545.59,211.75,211.75,15545.59,9,8,38.00,93.71,0.00,1.21
1017043,2021-10-26,Glenn,California,4406,42.00,6021,CA,County,28393,15517.91,147.92,147.92,15517.91,25,9,22.00,10.50,0.00,0.21
44968,2021-10-26,Madera,California,24218,285.00,6039,CA,County,157327,15393.42,181.15,181.15,15393.42,16,10,263.00,65.57,0.00,0.71


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
446297,2021-10-26,Imperial,California,36262,765.00,6025,CA,County,181215,20010.48,422.15,422.15,20010.48,1,3,191.00,58.50,0.00,0.29
3207,2021-10-26,Los Angeles,California,1487318,26579.00,6037,CA,County,10039107,14815.24,264.75,264.75,14815.24,2,13,845.00,1013.93,27.00,16.64
221521,2021-10-26,San Bernardino,California,360071,5754.00,6071,CA,County,2180085,16516.37,263.93,263.93,16516.37,3,5,1394.00,328.50,34.00,14.07
98717,2021-10-26,Stanislaus,California,86874,1341.00,6099,CA,County,550660,15776.34,243.53,243.53,15776.34,4,7,136.00,164.86,3.00,2.79
78455,2021-10-26,San Joaquin,California,103316,1745.00,6077,CA,County,762148,13555.90,228.96,228.96,13555.90,5,18,540.00,138.57,0.00,4.14
783850,2021-10-26,Inyo,California,2217,41.00,6027,CA,County,18039,12290.04,227.29,227.29,12290.04,6,23,75.00,23.29,0.00,0.14
951620,2021-10-26,Kings,California,33325,335.00,6031,CA,County,152940,21789.59,219.04,219.04,21789.59,7,2,387.00,89.43,2.00,1.86
1078304,2021-10-26,Tuolumne,California,7178,118.00,6109,CA,County,54478,13175.96,216.60,216.60,13175.96,8,22,27.00,25.36,1.00,0.86
709953,2021-10-26,Merced,California,43167,588.00,6047,CA,County,277680,15545.59,211.75,211.75,15545.59,9,8,38.00,93.71,0.00,1.21
122514,2021-10-26,Tulare,California,81147,986.00,6107,CA,County,466195,17406.24,211.50,211.50,17406.24,10,4,772.00,208.79,8.00,3.14


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,88.29
1,03/21/20,Imperial,4,2.21,3,0.00,80.86
2,03/22/20,Imperial,4,2.21,3,0.00,68.57
3,03/23/20,Imperial,4,2.21,3,0.00,68.57
4,03/24/20,Imperial,9,4.97,3,5.00,68.93
...,...,...,...,...,...,...,...
5810,10/22/21,Glenn,4384,15440.43,9,17.00,11.14
5811,10/23/21,Glenn,4384,15440.43,9,0.00,11.14
5812,10/24/21,Glenn,4384,15440.43,9,0.00,11.14
5813,10/25/21,Glenn,4384,15440.43,9,0.00,9.71


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.07
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.07
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.07
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.07
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.07
...,...,...,...,...,...,...,...
5916,10/22/21,Tulare,978.00,209.78,10,10.00,3.29
5917,10/23/21,Tulare,978.00,209.78,10,0.00,3.29
5918,10/24/21,Tulare,978.00,209.78,10,0.00,3.29
5919,10/25/21,Tulare,978.00,209.78,10,0.00,2.71


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)